#### Imports/setup

In [1]:
from parameters.config_imports import *

In [2]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Inputs : feature collection (without Geo IDs)
- Choose between 1) GEE asset or 2) Geojson input
- Comment out the one you dont want

In [15]:
# asset_id = "projects/ee-andyarnellgee/assets/example_asset" # asset id
# roi = ee.FeatureCollection(asset_id) # load feature collection asset

geojson_path = "input_examples/geojson_example.geojson" # geojson file
roi = geojson_to_ee(geojson_path) # convert geojson to feature collection

In [4]:
roi # view feature collection properties

In [5]:
# optional code: uncomment below line to test converting to point inputs using polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over 500 polygons currently this will go to Google Drive and require manual uploading here
- This value can be changed in parameters.config_runtime.py 
- NB Currently stats processing sent to a separate notebook 'process_whisp_stats.pynb'

In [6]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
Datasets present in lookup:  True
geo id not present ['Plot_area_ha', 'Geometry_type', 'Country', 'Centroid_lon', 'Centroid_lat', 'Unit']
...stats processing started
...ee to df processing time: 5.626198768615723 seconds


### Display it


In [7]:
df

,Plot_area_ha,Geometry_type,Country,Centroid_lon,Centroid_lat,Unit,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,...,TMF_def_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,1.938,Polygon,GHA,-1.611942,6.159539,ha,0.362,0.000,0.000,1.938,...,0.450,1.552,0.000,0.000,0,0,0.000,0,0.000,00000000000000000000
1,16.602,Polygon,GHA,-2.157144,5.981149,ha,16.602,14.493,15.792,16.602,...,0.000,0.000,0.000,0.000,0,0,0.000,0,0.973,00000000000000000001
2,31.211,Polygon,IDN,103.956096,-3.054668,ha,0.000,0.000,0.000,1.718,...,31.211,31.020,31.211,6.401,0,0,0.000,0,0.000,00000000000000000002
3,20.884,Polygon,IDN,103.977512,-3.083808,ha,5.842,0.000,6.472,20.099,...,3.081,9.043,0.000,0.000,0,0,0.000,0,0.000,00000000000000000003
4,8.279,Polygon,CIV,-4.101646,5.711935,ha,0.446,0.001,0.023,7.834,...,2.676,3.194,0.000,0.000,0,0,1.602,0,0.001,00000000000000000004
5,3.613,Polygon,CIV,-7.507022,6.071468,ha,1.485,0.000,0.000,3.613,...,0.958,1.955,0.000,0.000,0,0,0.000,0,0.000,00000000000000000005


#### Export it to CSV file
NB if exporting to Google Drive skip this and upload to your results folder 'whisp_output_table.csv'

In [8]:
#export output csv (default: '/home/sepal-user/module_results/whisp/whisp_output_table.csv')
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


#### Add Geo ids (optional)
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- If processing doesn't complete, just rerun this cell and it should pickup from where you left.
- If registering many features you can also click dollar sign in bottom right of Sepal and under 'Sessions' heading extend the length as needed.

In [9]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)

Chosen path for temp lookup csv:/home/sepal-user/module_results/whisp/temp_geo_id_lookup.csv 

Check passed: feature collection and table same size: 6 rows
Number without geo ids:0
All geo ids present in lookup csv
Done 



##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [10]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

new csv:  /home/sepal-user/module_results/whisp/whisp_output_table_w_geo_id.csv


##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [11]:
# remove_column_from_csv(
#     csv_file="whisp_output_table_w_geo_id.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [12]:
# Replace with name of your CSV file 
# NB if skipped 'Add Geo ids' should set to "out_directory/whisp_output_table.csv" 
csv_file_input =out_directory/'whisp_output_table_w_geo_id.csv'

# import csv as a dataframe
df_w_stats = pd.read_csv(csv_file_input)

df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [13]:
df_w_risk

,Plot_area_ha,Geo_id,Geometry_type,Country,Centroid_lon,Centroid_lat,Unit,EUFO_2020,GLAD_Primary,TMF_undist,...,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1.938,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,Polygon,GHA,-1.611942,6.159539,ha,0.362,0.000,0.000,...,0,0.000,0,0.000,0,yes,no,yes,no,low
1,16.602,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,Polygon,GHA,-2.157144,5.981149,ha,16.602,14.493,15.792,...,0,0.000,0,0.973,1,yes,no,yes,yes,low
2,31.211,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,Polygon,IDN,103.956096,-3.054668,ha,0.000,0.000,0.000,...,0,0.000,0,0.000,2,yes,yes,yes,no,low
3,20.884,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,Polygon,IDN,103.977512,-3.083808,ha,5.842,0.000,6.472,...,0,0.000,0,0.000,3,yes,no,yes,no,low
4,8.279,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,Polygon,CIV,-4.101646,5.711935,ha,0.446,0.001,0.023,...,0,1.602,0,0.001,4,yes,yes,yes,yes,low
5,3.613,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,Polygon,CIV,-7.507022,6.071468,ha,1.485,0.000,0.000,...,0,0.000,0,0.000,5,yes,yes,yes,no,low


##### Export table with risk columns to csv 

In [14]:
output_file=out_directory/'whisp_output_table_w_risk.csv' # edit as required

df_w_risk.to_csv(output_file)
